<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_GRAPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.2.0-rc1


In [4]:
class PropagatorVL:
    def __init__(self, no_of_steps, delta_t):
        self.delta_t = delta_t
        self.duration = no_of_steps * delta_t

        self.x = tf.constant(
            [[0, 1], [1, 0]], dtype=tf.complex128
        )
        self.y = tf.constant(
            [[0 + 0j, 0 - 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
        )
        
        """
            Van Loan generators for control robustness
        """
        xL = np.block([
            [self.x.numpy(), self.x.numpy()],
            [np.zeros((2, 2)), self.x.numpy()]          
        ])
        yL = np.block([
            [self.y.numpy(), self.y.numpy()],
            [np.zeros((2, 2)), self.y.numpy()]          
        ])

        self.generators =  tf.stack([
                                     tf.constant(xL, dtype=tf.complex128),
                                     tf.constant(yL, dtype=tf.complex128)
                                     ])

        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 2], dtype=tf.float64), dtype=tf.float64
        )


        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        regularized_amplitudes = 1 / np.sqrt(2) * tf.math.tanh(
            self.ctrl_amplitudes
        )

        exponents = tf.linalg.tensordot(
            tf.cast(regularized_amplitudes, dtype=tf.complex128),
            -2 * np.pi *(0 + 1j) * self.delta_t * self.generators, 1
        )
        return tf.linalg.expm(exponents)
    
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    @tf.function
    def target(self):
        propagator = self.propagate()
        tr = tf.linalg.trace(tf.linalg.matmul(self.x, propagator[0:2, 0:2]))
        """
            infidelity part in the target
        """
        infidelity = 1 - tf.math.real(tr * tf.math.conj(tr)) / (2 ** 2)
        """
            robustness term in the target
        """
        norm_squared = 1 / ((2 * np.pi * self.duration) ** 2) / 2 * (
            tf.math.real(
                tf.linalg.trace(
                    tf.linalg.matmul(
                        propagator[0:2, 3:4],
                        propagator[0:2, 3:4],
                        adjoint_b=True
                    )
                )
            )
        )

        return 0.5 * infidelity + 0.5 * norm_squared

propagatorVL = PropagatorVL(1000, 0.001)

optimizer = tf.keras.optimizers.Adam(0.01)

propagatorVL.ctrl_amplitudes.assign(
    tf.random.uniform([1000, 2], -1, 1, dtype=tf.float64)
)

def optimization_step():
    with tf.GradientTape() as tape:
        current_target = propagatorVL.target()
    gradients = tape.gradient(current_target, [propagatorVL.ctrl_amplitudes])
    optimizer.apply_gradients(zip(gradients, [propagatorVL.ctrl_amplitudes]))
    return current_target

steps = range(1000)
for step in steps:
    current_target = optimization_step()
    print('step %2d: target=%2.5f' %
          (step, current_target))
    
propagatorVL.ctrl_amplitudes.numpy()

step  0: target=0.49956
step  1: target=0.49808
step  2: target=0.49561
step  3: target=0.49215
step  4: target=0.48768
step  5: target=0.48217
step  6: target=0.47564
step  7: target=0.46806
step  8: target=0.45944
step  9: target=0.44978
step 10: target=0.43908
step 11: target=0.42737
step 12: target=0.41468
step 13: target=0.40105
step 14: target=0.38655
step 15: target=0.37121
step 16: target=0.35512
step 17: target=0.33837
step 18: target=0.32105
step 19: target=0.30327
step 20: target=0.28512
step 21: target=0.26674
step 22: target=0.24825
step 23: target=0.22979
step 24: target=0.21149
step 25: target=0.19346
step 26: target=0.17588
step 27: target=0.15884
step 28: target=0.14247
step 29: target=0.12688
step 30: target=0.11219
step 31: target=0.09847
step 32: target=0.08578
step 33: target=0.07420
step 34: target=0.06375
step 35: target=0.05444
step 36: target=0.04628
step 37: target=0.03924
step 38: target=0.03328
step 39: target=0.02834
step 40: target=0.02436
step 41: target=

array([[ 0.40119539, -1.2146701 ],
       [ 0.61587632, -1.17929686],
       [ 1.50761675, -1.67010281],
       ...,
       [ 1.13795295,  1.24769022],
       [ 0.10517093,  1.69986274],
       [ 0.81423235,  1.15582679]])